In [1]:
!pip install transformers datasets torch onnx onnxruntime

In [2]:
!pip install accelerate -U

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load dataset
dataset = load_dataset("imdb")

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='/content',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("/content/bert_imdb")
tokenizer.save_pretrained("/content/bert_imdb")


2024-07-14 08:26:32.372763: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 08:26:32.372820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 08:26:32.374421: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:14

Epoch,Training Loss,Validation Loss
1,0.251700,0.271707
2,0.147200,0.257475
3,0.076000,0.311451


('/content/bert_imdb/tokenizer_config.json',
 '/content/bert_imdb/special_tokens_map.json',
 '/content/bert_imdb/vocab.txt',
 '/content/bert_imdb/added_tokens.json')

In [6]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Load the trained model
model = BertForSequenceClassification.from_pretrained('/content/bert_imdb')
tokenizer = BertTokenizer.from_pretrained('/content/bert_imdb')

# Create dummy input
dummy_input = torch.randint(0, 100, (1, 512))

# Export the model to ONNX format with a higher opset version
torch.onnx.export(model,
                  dummy_input,
                  "bert_sentiment.onnx",
                  input_names=['input_ids'],
                  output_names=['output'],
                  opset_version=14,  # Use opset version 14
                  dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'}})


In [7]:
!pip install openvino-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 53.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 22.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.3 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [8]:
import numpy as np
from openvino.runtime import Core

# Load the ONNX model using OpenVINO
onnx_model_path = "bert_sentiment.onnx"
ie = Core()
model = ie.read_model(model=onnx_model_path)
compiled_model = ie.compile_model(model=model, device_name="CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

# Define a function to perform inference
def predict(text):
    inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    result = compiled_model([input_ids])[output_layer]
    return np.argmax(result)

# Test the model
text = "This movie is fantastic!"
prediction = predict(text)
print("Sentiment:", "Positive" if prediction == 1 else "Negative")


Sentiment: Positive
